In [ ]:
python evaluate.py  --resume ./results/train_offline/best_checkpoint.pth  --gpu-ids 1  --outdir results/val_offline --split test
python evaluate.py   --resume ./results/train_offline/best_checkpoint.pth   --gpu-ids 1   --outdir results/val_offline   --split test  --no-render


In [ ]:
python train_belfusion.py config=config/1_belfusion_vae.yaml name=All_VAEv2_W50
python train_belfusion.py config=config/2_belfusion_ldm.yaml name=Belfusion arch.args.k=10 arch.args.online=False

In [ ]:
# 0) 路径与清理
conda deactivate || true
conda env remove -p /transfer/conda/envs/react -y 2>/dev/null || true
export CONDA_ENVS_PATH=/transfer/conda/envs
export CONDA_PKGS_DIRS=/transfer/conda/pkgs
mkdir -p "$CONDA_ENVS_PATH" "$CONDA_PKGS_DIRS"

# 1) 新建环境（按你要的 py3.9）
conda create -p /transfer/conda/envs/react python=3.9 -y
conda activate /transfer/conda/envs/react

# 2) 先装“conda 版”的 PyTorch 2.0.1 + CUDA 11.8（关键！别用 pip）
conda install -y -c pytorch -c nvidia \
  pytorch=2.0.1 torchvision=0.15.2 torchaudio=2.0.2 pytorch-cuda=11.8

# 3) 安装 PyTorch3D 的依赖（从 conda-forge 拿 fvcore 和 iopath）
conda install -y -c conda-forge fvcore=0.1.5 iopath=0.1.10

# 4) 安装 PyTorch3D 0.7.4（pytorch3d 官方 channel）
conda install -y -c pytorch3d pytorch3d=0.7.4

# 5) 防止“同名遮蔽”（项目根别有 ./pytorch3d/ 目录）
cd /transfer/Master_Project/Offline_Multiple_Appropriate_Facial_Reaction/BeLFusion
[ -d pytorch3d ] && mv pytorch3d _pytorch3d_src_ignored

# 6) 验证
python - <<'PY'
import torch, pytorch3d
from pytorch3d.structures import Meshes
print("torch", torch.__version__, "cuda", torch.version.cuda)
print("pytorch3d from", pytorch3d.__file__)
PY


In [ ]:
# P1 -> Expert_video
find . -depth -type d -name P1 -print0 | while IFS= read -r -d '' d; do
  tgt="${d%/P1}/Expert_video"
  # 如果目标已存在且不是软链接，就跳过，防止覆盖真实目录
  if [ -e "$tgt" ] && [ ! -L "$tgt" ]; then
    echo "[skip] target exists: $tgt"
    continue
  fi
  # 若有残留同名软链接，先删
  [ -L "$tgt" ] && rm -f "$tgt"
  mv "$d" "$tgt"
  echo "[renamed] $d -> $tgt"
done

# P2 -> Novice_video
find . -depth -type d -name P2 -print0 | while IFS= read -r -d '' d; do
  tgt="${d%/P2}/Novice_video"
  if [ -e "$tgt" ] && [ ! -L "$tgt" ]; then
    echo "[skip] target exists: $tgt"
    continue
  fi
  [ -L "$tgt" ] && rm -f "$tgt"
  mv "$d" "$tgt"
  echo "[renamed] $d -> $tgt"
done


In [ ]:
python - <<'PY'
import os, yaml, pandas as pd, cv2, inspect
from dataset import ReactionDataset

# 1) 找数据根 & 打印前几行
cfg = yaml.safe_load(open('results/train_offline/config.yaml'))
root = cfg.get('dataset_path', '../dataset/data')   # 这里是 evaluate 用到的 dataset_path
csv = os.path.join(root, 'test.csv')       # 你现在用的是 --split test
print('root =', root)
print('csv  =', csv, 'exists=', os.path.exists(csv))
df = pd.read_csv(csv, header=None)
print('HEAD:\n', df.head(5).to_string(index=False))

# 2) 找到第一条两边都存在的样本
ok_idx = None
for i,(sp,lp,*_) in df[[0,1]].itertuples():
    sp, lp = str(sp), str(lp)
    if os.path.exists(sp) and os.path.exists(lp):
        ok_idx = i; break
    else:
        print('[skip missing]', i, os.path.exists(sp), sp, '|', os.path.exists(lp), lp)

if ok_idx is None:
    raise SystemExit("❌ test.csv 里没有一条两边路径都存在的样本。先修 CSV 或数据路径。")

print('[probe index]', ok_idx)
sp, lp = map(str, df.loc[ok_idx, [0,1]].tolist())

# 3) 分别试读 speaker / listener 的路径（视频或CSV）
def probe_path(p):
    p_lower = p.lower()
    if p_lower.endswith(('.mp4', '.avi', '.mov', '.mkv')):
        cap = cv2.VideoCapture(p)
        ok, frame = cap.read()
        print('VIDEO open:', p, 'ok=', ok, 'shape=' if ok else '', None if not ok else frame.shape)
        cap.release()
    elif p_lower.endswith('.csv'):
        t = pd.read_csv(p, nrows=3)
        print('CSV open:', p, 'shape=', t.shape, 'cols=', list(t.columns)[:5])
    else:
        print('FILE exists:', p, 'size=', os.path.getsize(p))

probe_path(sp)
probe_path(lp)

# 4) 直接构建 Dataset，拉一条样本（不用 DataLoader）
sig = inspect.signature(ReactionDataset.__init__)
print('ReactionDataset.__init__ signature:\n', sig)

# 猜测最小必需参数（和 evaluate 一致的关键开关）：
ds = ReactionDataset(
    root, 'test',
    img_size=cfg.get('img_size', 224), crop_size=cfg.get('crop_size', 224),
    load_audio=False, load_video_s=True, load_video_l=True,
    load_emotion_s=True, load_emotion_l=False  # 按你脚本里的 get_dataloader 传参习惯
)
print('len(ds)=', len(ds))
item = ds[ok_idx]
print('✅ got item keys:', item.keys() if isinstance(item, dict) else type(item))
PY


阶段 B：只算指标（从缓存读，超快）
/transfer/conda/envs/react/lib/python3.9/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
pred: torch.Size([32, 750, 25]) gt: torch.Size([32, 750, 25])
/transfer/Master_Project/Offline_Multiple_Appropriate_Facial_Reaction/BeLFusion/metric/FRC.py:27: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/transfer/Master_Project/Offline_Multiple_Appropriate_Facial_Reaction/BeLFusion/metric/FRC.py:28: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
FRC(local-KNN)= 0.12711794751420197

In [ ]:
python -u evaluate.py \
  --resume ./results/train_offline/best_checkpoint.pth \
  --gpu-ids 1 \
  --outdir results/val_offline \
  --split test \
  --dataset-path ../dataset/data \
  --no-render \
  --limit 32 -b 1 -j 0 \
  --save-preds-dir results/preds_test \
  --save-metrics-cache results/cache/test_limit32.npz \
  --skip-neighbor-metrics


In [ ]:
python compute_metrcs.py 

FRC  = 0.137000033561443
FRD  = 75.0037361297156
FRVar = 0.006847794633358717
FRDvs = 0.008508464321494102
S-MSE = 0.0
TLCC = 49.0

In [ ]:
import os, pandas as pd, numpy as np

root = "../dataset/data"
src  = os.path.join(root, "test.csv")
dst  = os.path.join(root, "test_diverse.csv")

def _str(p):
    return str(p).strip() if p is not None and not (isinstance(p,float) and np.isnan(p)) else ""

def exists_video(rel):
    rel = _str(rel)
    return bool(rel) and os.path.exists(os.path.join(root, rel))

def infer_listener(sp):
    sp = _str(sp)
    if "/Expert_video/" in sp:
        lp = sp.replace("/Expert_video/", "/Novice_video/")
    elif "/Novice_video/" in sp:
        lp = sp.replace("/Novice_video/", "/Expert_video/")
    else:
        return ""
    return lp if exists_video(lp) else ""

def emotion_csv_rel(rel):
    rel = _str(rel)
    return os.path.join(root, "test", "Emotion", f"{rel}.csv")

def has_emotion(rel):
    return os.path.exists(emotion_csv_rel(rel))

df0 = pd.read_csv(src, header=None)

# --- 自动定位列：优先使用第0行里带有 'speaker_path'/'listener_path' 的列 ---
sp_col = None
lp_col = None
for j in range(min(5, df0.shape[1])):  # 最多前5列里找
    v = str(df0.iloc[0, j]).lower()
    if "speaker_path" in v and sp_col is None: sp_col = j
    if "listener_path" in v and lp_col is None: lp_col = j

if sp_col is None or lp_col is None:
    # 后备：常见结构是 [index, speaker, listener]
    if df0.shape[1] >= 3:
        sp_col, lp_col = 1, 2
    elif df0.shape[1] == 2:
        sp_col, lp_col = 0, 1
    else:
        raise RuntimeError(f"CSV列不足，shape={df0.shape}")

# 丢掉表头行，取两列
df = df0.iloc[1:, [sp_col, lp_col]].copy()
df.columns = ["speaker_path", "listener_path"]

# 统一转字符串，去空白
df["speaker_path"] = df["speaker_path"].map(_str)
df["listener_path"] = df["listener_path"].map(_str)

# 用 speaker 推断缺失的 listener
miss = df["listener_path"].eq("") | df["listener_path"].isna()
df.loc[miss, "listener_path"] = df.loc[miss, "speaker_path"].map(infer_listener)

# 过滤：两边视频都存在
ok = df["speaker_path"].map(exists_video) & df["listener_path"].map(exists_video)
df = df[ok].copy()

# 再过滤：两边 Emotion CSV 都存在
ok2 = df["speaker_path"].map(has_emotion) & df["listener_path"].map(has_emotion)
df = df[ok2].copy()

# 去重
df = df.drop_duplicates().reset_index(drop=True)

# 抽样更分散（固定种子），比如 200 条
np.random.seed(42)
n = min(200, len(df))
idx = np.linspace(0, len(df)-1, num=n, dtype=int) if n>0 else np.array([], dtype=int)
sub = df.iloc[idx].reset_index(drop=True)

# 写回：加表头行
header = pd.DataFrame([["", "speaker_path", "listener_path"]])
out = pd.concat([header, sub], axis=0)
out.to_csv(dst, header=False, index=False, encoding="utf-8")

print("wrote:", dst, "rows(without header)=", len(sub))
print(sub.head(5))